In [ ]:
import tkinter as tk
import tkinter.font as font
from tkinter import ttk
import numpy as np
import copy
import random
import math





class MineSweeper():
    
    #-----------------------------------------------------------------------------------------
    #Initialization Functions
    #-----------------------------------------------------------------------------------------
    def __init__(self):
        

        self.window = tk.Tk()
        self.window.title("Minesweeper")
        self.MainFrame = ttk.Frame(self.window)
        
        self.playArea = ttk.Frame(self.MainFrame)
        self.playArea.grid(row = 1,column = 0, columnspan = 3)
        
        self.difficulty = 'Beginner'
        self.tiles = []
        
        #self.newGameButton = ttk.Button(self.MainFrame, text = "New Game", command = self.newGame)
        #self.newGameButton.grid(row = 0,column = 0)
        
        self.mineCounterVar = tk.IntVar()
        self.mineCounter = tk.Label(self.MainFrame, textvariable = self.mineCounterVar, relief = 'groove')
        self.mineCounter.grid(row = 0, column = 0)
        
        self.GameLabel = tk.Label(self.MainFrame, text = "Minesweeper", relief = "raised")
        self.GameLabel.grid(row = 0, column = 1)
        self.GameLabel.bind("<Button-1>", self.newGame)
        
        
        self.difficultyOptions = ['Beginner', 'Intermediate', 'Expert']
        
        
        self.gameDifficultyddVar = tk.StringVar()
        self.gameDifficultyddVar.trace('w', self.DifficultySelect)
        self.gameDifficultydd = ttk.OptionMenu(self.MainFrame, self.gameDifficultyddVar, self.difficultyOptions[0], *self.difficultyOptions)
        self.gameDifficultydd.grid(row = 0, column = 2)
        self.gameDifficultyddVar.set(self.difficultyOptions[0])
        
        self.MainFrame.grid()
        
        #Input from User
        self.clickClear = False
        self.gameover = False
        
        self.newGame()
        
    def newGame(self, *args):
        for tile in self.tiles:
            tile.destroy()
        self.initializeBoard()
        self.firstClick = True
        
    def DifficultySelect(self, *args):
        self.difficulty = self.gameDifficultyddVar.get()
        self.newGame()
        
    def mainloop(self):
        self.window.mainloop()
                
    #-----------------------------------------------------------------------------------------
    #Drawing Functions
    #The Modules required to draw required game based object on canvas
    #-----------------------------------------------------------------------------------------
    
    def initializeBoard(self):
        if self.difficulty == 'Beginner':
            self.dim = [7,7]
            self.num_of_mines = 10
        elif self.difficulty == 'Intermediate':
            self.dim = [16,16]
            self.num_of_mines = 40
            
        elif self.difficulty == 'Expert':
            self.dim = [16,30]
            self.num_of_mines = 100
        
        self.mineCounterVar.set(self.num_of_mines)
        
        self.mineField = np.zeros(self.dim)
        self.markedMines = []
        self.mineLocations = []
        while len(self.mineLocations)<self.num_of_mines:
            possibleLocation = random.randint(1,self.dim[0]*self.dim[1]-1)
            if possibleLocation not in self.mineLocations:
                self.mineLocations.append(possibleLocation)
        
        
        
        
        self.tiles = []
        buttonFont = tk.font.Font(family='Helvetica', size=16, weight='bold')
        for i in range(self.dim[0]):
            for j in range(self.dim[1]):
                tile = tk.Label(self.playArea, relief = 'raised', font = buttonFont, text = ' ', width = 2)
                tile.grid(row = i, column=j)
                m = i*self.dim[1]+j
                tile.bind("<Button-1>", lambda event, obj=tile: self.LabelClicked(event,obj))
                tile.bind("<Button-3>", lambda event, obj=tile: self.LabelRightClicked(event,obj))
                self.tiles.append(tile)
                
                if i*self.dim[1]+j in self.mineLocations:
                    self.mineField[i][j] = 10
                else:
                    if i == 0:
                        rowCheckRange = [0,1]
                    elif i == self.dim[0]-1:
                        rowCheckRange = [-1,0]
                    else:
                        rowCheckRange = [-1,0,1]
                    if j == 0:
                        colCheckRange = [0,1]
                    elif j == self.dim[1]-1:
                        colCheckRange = [-1,0]
                    else:
                        colCheckRange = [-1,0,1]
                        
                    for rowCheck in rowCheckRange:
                        for colCheck in colCheckRange:
                            if (i+rowCheck)*self.dim[1]+(j+colCheck) in self.mineLocations:
                                    self.mineField[i][j] = self.mineField[i][j]+1
                            
        

    def display_gameover(self):
        self.playArea.delete("all")
        self.playArea.create_text(
            size_of_board / 2,
            3 * size_of_board / 8,
            font="cmr 40 bold",
            fill="Green",
            text="Game Over",
        )
    #----------------------------------------------------------------------------------------
    #Logical Function
    #The modules required to carry out the game functio
    #----------------------------------------------------------------------------------------
    
    def LabelClicked(self, event, obj):

        m = self.tiles.index(obj)
        
        
        
        if self.firstClick == True:
            
            if m in self.mineLocations:
                k=0
                clear = False
                while not clear:
                    if k not in self.mineLocations:
                        
                        self.mineLocations[self.mineLocations.index(m)] = k
                        self.mineField[int(np.floor(k/self.dim[1]))][k%self.dim[1]] = 10
                        self.mineField[int(np.floor(m/self.dim[1]))][m%self.dim[1]] = 0
                        clear = True
                    k = k+1
                    
                for k in range(len(self.tiles)):
                    if k in self.mineLocations:
                        self.mineField[int(np.floor(k/self.dim[1]))][k%self.dim[1]] = 10
                    else:
                        if int(np.floor(k/self.dim[1])) == 0:
                            rowCheckRange = [0,1]
                        elif int(np.floor(k/self.dim[1])) == self.dim[0]-1:
                            rowCheckRange = [-1,0]
                        else:
                            rowCheckRange = [-1,0,1]
                        if k%self.dim[1] == 0:
                            colCheckRange = [0,1]
                        elif k%self.dim[1] == self.dim[1]-1:
                            colCheckRange = [-1,0]
                        else:
                            colCheckRange = [-1,0,1]

                        self.mineField[int(np.floor(k/self.dim[1]))][k%self.dim[1]] = 0
                        for rowCheck in rowCheckRange:
                            for colCheck in colCheckRange:

                                if (np.floor(k/self.dim[1])+rowCheck)*self.dim[1]+(k%self.dim[1]+colCheck) in self.mineLocations:
                                    self.mineField[int(np.floor(k/self.dim[1]))][k%self.dim[1]] = self.mineField[int(np.floor(k/self.dim[1]))][k%self.dim[1]]+1
                        
                        
            
            self.firstClick = False
            self.changeTile(m)
            
        elif obj["relief"] == "groove":
            if int(np.floor(m/self.dim[1])) == 0:
                rowCheckRange = [0,1]
            elif int(np.floor(m/self.dim[1])) == self.dim[0]-1:
                rowCheckRange = [-1,0]
            else:
                rowCheckRange = [-1,0,1]
            if m%self.dim[1] == 0:
                colCheckRange = [0,1]
            elif m%self.dim[1] == self.dim[1]-1:
                colCheckRange = [-1,0]
            else:
                colCheckRange = [-1,0,1]

            self.minesFound = 0
            
            for rowCheck in rowCheckRange:
                for colCheck in colCheckRange:

                    if (np.floor(m/self.dim[1])+rowCheck)*self.dim[1]+(m%self.dim[1]+colCheck) in self.markedMines:
                        self.minesFound = self.minesFound+1
            
            if self.minesFound == self.mineField[int(np.floor(m/self.dim[1]))][m%self.dim[1]]:
                for rowCheck in rowCheckRange:
                    for colCheck in colCheckRange:
                        mprime = m+colCheck+rowCheck*self.dim[1]
                        if mprime not in self.markedMines:
                            
                            self.changeTile(mprime)
                            
        else:
            self.changeTile(m)

                    
        
        self.checkWin()
        
    def LabelRightClicked(self, event, obj):
        if obj['relief'] == 'raised':
            m = self.tiles.index(obj)
            if self.tiles[m]["text"] == ' ':
                self.markedMines.append(m)
                self.tiles[m]["text"] = 'X'
                self.mineCounterVar.set(self.mineCounterVar.get()-1)
                
            elif self.tiles[m]["text"] == 'X':
                self.markedMines.remove(m)
                self.tiles[m]["text"] = ' '
                self.mineCounterVar.set(self.mineCounterVar.get()+1)
                
        
        
    def changeTile(self,tile):
        self.tiles[tile]["relief"] = "groove"
        if self.mineField[int(np.floor(tile/self.dim[1]))][tile%self.dim[1]] == 0:
            if not self.clickClear:
                self.emptyTile(tile)
            else:
                self.tiles[tile]["text"] = ' ' 
        elif self.mineField[int(np.floor(tile/self.dim[1]))][tile%self.dim[1]] == 1:
            self.tiles[tile]["text"] = '1'    
            self.tiles[tile]["fg"] = 'blue'
        elif self.mineField[int(np.floor(tile/self.dim[1]))][tile%self.dim[1]] == 2:
            self.tiles[tile]["text"] = '2'
            self.tiles[tile]["fg"] = 'green'
        elif self.mineField[int(np.floor(tile/self.dim[1]))][tile%self.dim[1]] == 3:
            self.tiles[tile]["text"] = '3'
            self.tiles[tile]["fg"] = 'red'
        elif self.mineField[int(np.floor(tile/self.dim[1]))][tile%self.dim[1]] == 4:
            self.tiles[tile]["text"] = '4' 
            self.tiles[tile]["fg"] = '#27285C'
        elif self.mineField[int(np.floor(tile/self.dim[1]))][tile%self.dim[1]] == 5:
            self.tiles[tile]["text"] = '5'  
            self.tiles[tile]["fg"] = '#800500'
        elif self.mineField[int(np.floor(tile/self.dim[1]))][tile%self.dim[1]] == 6:
            self.tiles[tile]["text"] = '6'
            self.tiles[tile]["fg"] = '#00A3E1'
        elif self.mineField[int(np.floor(tile/self.dim[1]))][tile%self.dim[1]] == 7:
            self.tiles[tile]["text"] = '7'
            self.tiles[tile]["fg"] = 'black'
        elif self.mineField[int(np.floor(tile/self.dim[1]))][tile%self.dim[1]] == 8:
            self.tiles[tile]["text"] = '8'
            self.tiles[tile]["fg"] = 'gray'
            
        elif self.mineField[int(np.floor(tile/self.dim[1]))][tile%self.dim[1]] == 10:
            for k in range(len(self.tiles)):
                if self.mineField[int(np.floor(k/self.dim[1]))][k%self.dim[1]] == 10:
                    self.tiles[k]["text"] = '*'
                    self.tiles[k]["fg"] = 'black'
                    self.tiles[k]["relief"] = "groove" 
            
    def emptyTile(self,tile):
        
        self.tiles[tile]["text"] = ' '
        self.emptySpaces = [tile]
        self.newEmptySpaces = [tile]
        
        while self.newEmptySpaces != []:
            for space in self.newEmptySpaces:
                
                m = space     
                if int(np.floor(m/self.dim[1])) == 0:
                    rowCheckRange = [0,1]
                elif int(np.floor(m/self.dim[1])) == self.dim[0]-1:
                    rowCheckRange = [-1,0]
                else:
                    rowCheckRange = [-1,0,1]
                if m%self.dim[1] == 0:
                    colCheckRange = [0,1]
                elif m%self.dim[1] == self.dim[1]-1:
                    colCheckRange = [-1,0]
                else:
                    colCheckRange = [-1,0,1]

                self.minesFound = 0

                for rowCheck in rowCheckRange:
                    for colCheck in colCheckRange:
                        if self.mineField[int(np.floor(m/self.dim[1]))+rowCheck][m%self.dim[1]+colCheck] == 0 and m+colCheck+rowCheck*self.dim[1] not in self.emptySpaces:
                            self.clickClear = True
                            self.changeTile(m+colCheck+rowCheck*self.dim[1])
                            self.clickClear = False
                            self.newEmptySpaces.append(m+colCheck+rowCheck*self.dim[1])
                            
                            
                self.emptySpaces.append(space)
                self.newEmptySpaces.remove(space)
                
        for space in self.emptySpaces:
                
                m = space     
                if int(np.floor(m/self.dim[1])) == 0:
                    rowCheckRange = [0,1]
                elif int(np.floor(m/self.dim[1])) == self.dim[0]-1:
                    rowCheckRange = [-1,0]
                else:
                    rowCheckRange = [-1,0,1]
                if m%self.dim[1] == 0:
                    colCheckRange = [0,1]
                elif m%self.dim[1] == self.dim[1]-1:
                    colCheckRange = [-1,0]
                else:
                    colCheckRange = [-1,0,1]

                self.minesFound = 0

                for rowCheck in rowCheckRange:
                    for colCheck in colCheckRange:
                        
                        self.clickClear = True
                        self.changeTile(m+colCheck+rowCheck*self.dim[1])
                        self.clickClear = False
                            
            
                
                
            
    def checkWin(self):
        if not self.gameover:
            clears = 0
            for tile in self.tiles:
                if tile['relief'] == 'groove':
                    clears = clears+1
            if clears == len(self.tiles)-self.num_of_mines:
                self.GameLabel['text'] = 'Complete!'
                


game = MineSweeper()
game.mainloop()

141

['seven', 'hundred', 'twenty', 'three']


723

True
False
